In [1]:
import re
import string
import scipy
import pickle
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from prettytable import PrettyTable
from astropy.table import Table , Column

In [2]:
trainData = pd.read_csv('BrainStrokeTrain.csv') 
testData = pd.read_csv('BrainStrokeTest.csv')

In [3]:
trainData

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
5,Male,74,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
6,Female,69,0,0,No,Private,Urban,94.39,22.8,never smoked,1
7,Female,78,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1
8,Female,81,1,0,Yes,Private,Rural,80.43,29.7,never smoked,1
9,Female,61,0,1,Yes,Govt_job,Rural,120.46,36.8,smokes,1


In [4]:
testData

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,71,0,0,Yes,Private,Urban,102.87,27.2,formerly smoked,1
1,Male,80,0,0,Yes,Self-employed,Rural,104.12,23.5,never smoked,1
2,Female,65,0,0,Yes,Private,Rural,100.98,28.2,formerly smoked,1
3,Male,69,0,1,Yes,Self-employed,Urban,195.23,28.3,smokes,1
4,Female,37,0,0,Yes,Private,Rural,73.50,26.1,formerly smoked,0
5,Female,40,0,0,Yes,Private,Rural,95.04,42.4,never smoked,0
6,Male,35,0,0,No,Private,Rural,85.37,33.0,never smoked,0
7,Female,20,0,0,No,Private,Urban,84.62,19.7,smokes,0


In [5]:
trainData.columns.name="index"
print("Train Dataset:")
print(trainData)

Train Dataset:
index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   67             0              1          Yes        Private   
1        Male   80             0              1          Yes        Private   
2      Female   49             0              0          Yes        Private   
3      Female   79             1              0          Yes  Self-employed   
4        Male   81             0              0          Yes        Private   
5        Male   74             1              1          Yes        Private   
6      Female   69             0              0           No        Private   
7      Female   78             0              0          Yes        Private   
8      Female   81             1              0          Yes        Private   
9      Female   61             0              1          Yes       Govt_job   
10     Female   54             0              0          Yes        Private   
11     Female   79             0     

In [6]:
print("Train Data Set Columns:")
trainDatadf=pd.DataFrame(trainData)
trainDataIndex=trainDatadf.columns
print(trainDataIndex)

print("\n")
print("Number of instances in Train Dataset")
print("Train Instances: %s"% (len(trainDatadf.index)))

Train Data Set Columns:
Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object', name='index')


Number of instances in Train Dataset
Train Instances: 20


In [7]:
testData.columns.name="index"
print("Test Dataset:")
print(testData)

Test Dataset:
index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   71             0              0          Yes        Private   
1        Male   80             0              0          Yes  Self-employed   
2      Female   65             0              0          Yes        Private   
3        Male   69             0              1          Yes  Self-employed   
4      Female   37             0              0          Yes        Private   
5      Female   40             0              0          Yes        Private   
6        Male   35             0              0           No        Private   
7      Female   20             0              0           No        Private   

index Residence_type  avg_glucose_level   bmi   smoking_status  stroke  
0              Urban             102.87  27.2  formerly smoked       1  
1              Rural             104.12  23.5     never smoked       1  
2              Rural             100.98  28.2  formerly

# Pre-Process Data

In [8]:
print("Train Dataset before Preprocessing: \n")
print(trainDatadf)
print ("\n")
print("Train Dataset after Preprocessing: \n")
preprocessTrainDatadf=trainDatadf.round({"height":2})
print(preprocessTrainDatadf)

Train Dataset before Preprocessing: 

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   67             0              1          Yes        Private   
1        Male   80             0              1          Yes        Private   
2      Female   49             0              0          Yes        Private   
3      Female   79             1              0          Yes  Self-employed   
4        Male   81             0              0          Yes        Private   
5        Male   74             1              1          Yes        Private   
6      Female   69             0              0           No        Private   
7      Female   78             0              0          Yes        Private   
8      Female   81             1              0          Yes        Private   
9      Female   61             0              1          Yes       Govt_job   
10     Female   54             0              0          Yes        Private   
11     Female 

In [9]:
print("Test Data Set Columns:")
testDatadf=pd.DataFrame(testData)
testDataIndex=testDatadf.columns
print(testDataIndex)

print("\n")
print("Number of instances in Test Dataset")
print("Test Instances: %s"% (len(testDatadf.index)))

Test Data Set Columns:
Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object', name='index')


Number of instances in Test Dataset
Test Instances: 8


In [10]:
print("Test Dataset before Preprocessing: \n")
print(testDatadf)
print ("\n")
print("Test Dataset after Preprocessing: \n")
preprocessTestDatadf=testDatadf.round({"height":2})
print(preprocessTestDatadf)

Test Dataset before Preprocessing: 

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   71             0              0          Yes        Private   
1        Male   80             0              0          Yes  Self-employed   
2      Female   65             0              0          Yes        Private   
3        Male   69             0              1          Yes  Self-employed   
4      Female   37             0              0          Yes        Private   
5      Female   40             0              0          Yes        Private   
6        Male   35             0              0           No        Private   
7      Female   20             0              0           No        Private   

index Residence_type  avg_glucose_level   bmi   smoking_status  stroke  
0              Urban             102.87  27.2  formerly smoked       1  
1              Rural             104.12  23.5     never smoked       1  
2              Rural            

# Label Encoding for Train/Test Data

In [11]:
print("Gender attribute encoding in Train Dataset: \n")

preprocessTrainDatadf["encode_gender"]=LabelEncoder().fit_transform(preprocessTrainDatadf["gender"])
print(preprocessTrainDatadf[["gender","encode_gender"]])

Gender attribute encoding in Train Dataset: 

index  gender  encode_gender
0        Male              1
1        Male              1
2      Female              0
3      Female              0
4        Male              1
5        Male              1
6      Female              0
7      Female              0
8      Female              0
9      Female              0
10     Female              0
11     Female              0
12     Female              0
13     Female              0
14       Male              1
15     Female              0
16     Female              0
17     Female              0
18     Female              0
19     Female              0


In [12]:
print("ever_married attribute encoding in train Dataset: \n")

preprocessTrainDatadf["encode_ever_married"]=LabelEncoder().fit_transform(preprocessTrainDatadf["ever_married"])
print(preprocessTrainDatadf[["ever_married","encode_ever_married"]])

ever_married attribute encoding in train Dataset: 

index ever_married  encode_ever_married
0              Yes                    1
1              Yes                    1
2              Yes                    1
3              Yes                    1
4              Yes                    1
5              Yes                    1
6               No                    0
7              Yes                    1
8              Yes                    1
9              Yes                    1
10             Yes                    1
11             Yes                    1
12             Yes                    1
13             Yes                    1
14             Yes                    1
15             Yes                    1
16             Yes                    1
17             Yes                    1
18              No                    0
19             Yes                    1


In [13]:
print("work_type attribute encoding in train Dataset: \n")

preprocessTrainDatadf["encode_work_type"]=LabelEncoder().fit_transform(preprocessTrainDatadf["work_type"])
print(preprocessTrainDatadf[["work_type","encode_work_type"]])

work_type attribute encoding in train Dataset: 

index      work_type  encode_work_type
0            Private                 1
1            Private                 1
2            Private                 1
3      Self-employed                 2
4            Private                 1
5            Private                 1
6            Private                 1
7            Private                 1
8            Private                 1
9           Govt_job                 0
10           Private                 1
11     Self-employed                 2
12           Private                 1
13           Private                 1
14           Private                 1
15           Private                 1
16           Private                 1
17          Govt_job                 0
18           Private                 1
19           Private                 1


In [14]:
print("Residence_type attribute encoding in train Dataset: \n")

preprocessTrainDatadf["encode_Residence_type"]=LabelEncoder().fit_transform(preprocessTrainDatadf["Residence_type"])
print(preprocessTrainDatadf[["Residence_type","encode_Residence_type"]])

Residence_type attribute encoding in train Dataset: 

index Residence_type  encode_Residence_type
0              Urban                      1
1              Rural                      0
2              Urban                      1
3              Rural                      0
4              Urban                      1
5              Rural                      0
6              Urban                      1
7              Urban                      1
8              Rural                      0
9              Rural                      0
10             Urban                      1
11             Urban                      1
12             Rural                      0
13             Rural                      0
14             Urban                      1
15             Urban                      1
16             Urban                      1
17             Rural                      0
18             Rural                      0
19             Rural                      0


In [15]:
print("smoking_status attribute encoding in train Dataset: \n")

preprocessTrainDatadf["encode_smoking_status"]=LabelEncoder().fit_transform(preprocessTrainDatadf["smoking_status"])
print(preprocessTrainDatadf[["smoking_status","encode_smoking_status"]])

smoking_status attribute encoding in train Dataset: 

index   smoking_status  encode_smoking_status
0      formerly smoked                      1
1         never smoked                      2
2               smokes                      3
3         never smoked                      2
4      formerly smoked                      1
5         never smoked                      2
6         never smoked                      2
7              Unknown                      0
8         never smoked                      2
9               smokes                      3
10              smokes                      3
11              smokes                      3
12             Unknown                      0
13              smokes                      3
14     formerly smoked                      1
15        never smoked                      2
16        never smoked                      2
17              smokes                      3
18        never smoked                      2
19        never smoked    

In [16]:
print("Gender attribute encoding in Train Dataset: \n")

preprocessTestDatadf["encode_gender"]=LabelEncoder().fit_transform(preprocessTestDatadf["gender"])
print(preprocessTestDatadf[["gender","encode_gender"]])

Gender attribute encoding in Train Dataset: 

index  gender  encode_gender
0        Male              1
1        Male              1
2      Female              0
3        Male              1
4      Female              0
5      Female              0
6        Male              1
7      Female              0


In [17]:
print("ever_married attribute encoding in train Dataset: \n")

preprocessTestDatadf["encode_ever_married"]=LabelEncoder().fit_transform(preprocessTestDatadf["ever_married"])
print(preprocessTestDatadf[["ever_married","encode_ever_married"]])

ever_married attribute encoding in train Dataset: 

index ever_married  encode_ever_married
0              Yes                    1
1              Yes                    1
2              Yes                    1
3              Yes                    1
4              Yes                    1
5              Yes                    1
6               No                    0
7               No                    0


In [18]:
print("work_type attribute encoding in train Dataset: \n")

preprocessTestDatadf["encode_work_type"]=LabelEncoder().fit_transform(preprocessTestDatadf["work_type"])
print(preprocessTestDatadf[["work_type","encode_work_type"]])

work_type attribute encoding in train Dataset: 

index      work_type  encode_work_type
0            Private                 0
1      Self-employed                 1
2            Private                 0
3      Self-employed                 1
4            Private                 0
5            Private                 0
6            Private                 0
7            Private                 0


In [19]:
print("Residence_type attribute encoding in train Dataset: \n")

preprocessTestDatadf["encode_Residence_type"]=LabelEncoder().fit_transform(preprocessTestDatadf["Residence_type"])
print(preprocessTestDatadf[["Residence_type","encode_Residence_type"]])

Residence_type attribute encoding in train Dataset: 

index Residence_type  encode_Residence_type
0              Urban                      1
1              Rural                      0
2              Rural                      0
3              Urban                      1
4              Rural                      0
5              Rural                      0
6              Rural                      0
7              Urban                      1


In [20]:
print("smoking_status attribute encoding in train Dataset: \n")

preprocessTestDatadf["encode_smoking_status"]=LabelEncoder().fit_transform(preprocessTestDatadf["smoking_status"])
print(preprocessTestDatadf[["smoking_status","encode_smoking_status"]])

smoking_status attribute encoding in train Dataset: 

index   smoking_status  encode_smoking_status
0      formerly smoked                      0
1         never smoked                      1
2      formerly smoked                      0
3               smokes                      2
4      formerly smoked                      0
5         never smoked                      1
6         never smoked                      1
7               smokes                      2


In [21]:
print("Orignal Training Dataset: \n")
print(trainData)

print("\n")

print("Training Dataset after Label Encoding: \n")

labelTrainDatadf= preprocessTrainDatadf[["encode_gender","age","hypertension","heart_disease","encode_ever_married","encode_work_type","encode_Residence_type","avg_glucose_level","bmi","encode_smoking_status","stroke"]]
fTrainDatadf= labelTrainDatadf.rename(columns={"encode_gender":"gender", "encode_ever_married":"ever_married","encode_work_type":"work_type","encode_Residence_type":"Residence_type","encode_smoking_status":"smoking_status"})
print(fTrainDatadf)

Orignal Training Dataset: 

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   67             0              1          Yes        Private   
1        Male   80             0              1          Yes        Private   
2      Female   49             0              0          Yes        Private   
3      Female   79             1              0          Yes  Self-employed   
4        Male   81             0              0          Yes        Private   
5        Male   74             1              1          Yes        Private   
6      Female   69             0              0           No        Private   
7      Female   78             0              0          Yes        Private   
8      Female   81             1              0          Yes        Private   
9      Female   61             0              1          Yes       Govt_job   
10     Female   54             0              0          Yes        Private   
11     Female   79      

In [22]:
print("Orignal Testing Dataset: \n")
print(testData)

print("\n")

print("Testing Dataset after Label Encoding: \n")

labelTestDatadf= preprocessTestDatadf[["encode_gender","age","hypertension","heart_disease","encode_ever_married","encode_work_type","encode_Residence_type","avg_glucose_level","bmi","encode_smoking_status","stroke"]]
fTestDatadf= labelTestDatadf.rename(columns={"encode_gender":"gender", "encode_ever_married":"ever_married","encode_work_type":"work_type","encode_Residence_type":"Residence_type","encode_smoking_status":"smoking_status"})
print(fTestDatadf)

Orignal Testing Dataset: 

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   71             0              0          Yes        Private   
1        Male   80             0              0          Yes  Self-employed   
2      Female   65             0              0          Yes        Private   
3        Male   69             0              1          Yes  Self-employed   
4      Female   37             0              0          Yes        Private   
5      Female   40             0              0          Yes        Private   
6        Male   35             0              0           No        Private   
7      Female   20             0              0           No        Private   

index Residence_type  avg_glucose_level   bmi   smoking_status  stroke  
0              Urban             102.87  27.2  formerly smoked       1  
1              Rural             104.12  23.5     never smoked       1  
2              Rural             100.98  2

# Train ML Algorithms using Train Data

In [52]:
inputTrainAt=fTrainDatadf.iloc[:, :-1].values
outputTrainAt=fTrainDatadf.iloc[:, 10].values

inputTestAt=fTestDatadf.iloc[:,: -1].values
outputTestAt=fTestDatadf.iloc[:, 10].values

In [53]:
inputTrainAt

array([[  1.  ,  67.  ,   0.  ,   1.  ,   1.  ,   1.  ,   1.  , 228.69,
         36.6 ,   1.  ],
       [  1.  ,  80.  ,   0.  ,   1.  ,   1.  ,   1.  ,   0.  , 105.92,
         32.5 ,   2.  ],
       [  0.  ,  49.  ,   0.  ,   0.  ,   1.  ,   1.  ,   1.  , 171.23,
         34.4 ,   3.  ],
       [  0.  ,  79.  ,   1.  ,   0.  ,   1.  ,   2.  ,   0.  , 174.12,
         24.  ,   2.  ],
       [  1.  ,  81.  ,   0.  ,   0.  ,   1.  ,   1.  ,   1.  , 186.21,
         29.  ,   1.  ],
       [  1.  ,  74.  ,   1.  ,   1.  ,   1.  ,   1.  ,   0.  ,  70.09,
         27.4 ,   2.  ],
       [  0.  ,  69.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,  94.39,
         22.8 ,   2.  ],
       [  0.  ,  78.  ,   0.  ,   0.  ,   1.  ,   1.  ,   1.  ,  58.57,
         24.2 ,   0.  ],
       [  0.  ,  81.  ,   1.  ,   0.  ,   1.  ,   1.  ,   0.  ,  80.43,
         29.7 ,   2.  ],
       [  0.  ,  61.  ,   0.  ,   1.  ,   1.  ,   0.  ,   0.  , 120.46,
         36.8 ,   3.  ],
       [  0.  ,  54.  ,   0.  

In [24]:
print("Prameters and their values: \n")
logisticRegression=LogisticRegression()
print(logisticRegression)

Prameters and their values: 

LogisticRegression()


In [25]:
logisticRegression.fit(inputTrainAt,outputTrainAt)

outputPredictionLR=logisticRegression.predict(inputTestAt)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
print("Prameters and their values: \n")
randomForest=RandomForestClassifier()
print(randomForest)

Prameters and their values: 

RandomForestClassifier()


In [27]:
randomForest.fit(inputTrainAt,outputTrainAt)
outputPredictionRF = randomForest.predict(inputTestAt)

In [28]:
print("Prameters and their values: \n")
linearSvc=LinearSVC()
print(linearSvc)

Prameters and their values: 

LinearSVC()


In [29]:
linearSvc.fit(inputTrainAt,outputTrainAt)
outputPredictionSVC = linearSvc.predict(inputTestAt)

D:\Anaconda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [30]:
print("Prameters and their values: \n")
bernoulliNB=BernoulliNB()
print(bernoulliNB)

Prameters and their values: 

BernoulliNB()


In [31]:
bernoulliNB.fit(inputTrainAt,outputTrainAt)
outputPredictionNB = bernoulliNB.predict(inputTestAt)

# Evaluate ML Algorithms using Test Data

In [32]:
print("prediction using Logistic Regression:\n")

testingLRdf = testData 
testingLRdf['predicted_stroke']=outputPredictionLR
print(testingLRdf)
lrAccuracy = format( logisticRegression.score(inputTestAt, outputTestAt))
print("\n")
print("Accuracy: %s"% lrAccuracy)

prediction using Logistic Regression:

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   71             0              0          Yes        Private   
1        Male   80             0              0          Yes  Self-employed   
2      Female   65             0              0          Yes        Private   
3        Male   69             0              1          Yes  Self-employed   
4      Female   37             0              0          Yes        Private   
5      Female   40             0              0          Yes        Private   
6        Male   35             0              0           No        Private   
7      Female   20             0              0           No        Private   

index Residence_type  avg_glucose_level   bmi   smoking_status  stroke  \
0              Urban             102.87  27.2  formerly smoked       1   
1              Rural             104.12  23.5     never smoked       1   
2              Rural       

In [33]:
print("prediction usinsg Random Forest Classifier:\n")

testingRFdf = testData 
testingRFdf['predicted_stroke']=outputPredictionRF
print(testingRFdf)
rfAccuracy = format( randomForest.score(inputTestAt, outputTestAt))
print("\n")
print("Accuracy: %s"% rfAccuracy)

prediction usinsg Random Forest Classifier:

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   71             0              0          Yes        Private   
1        Male   80             0              0          Yes  Self-employed   
2      Female   65             0              0          Yes        Private   
3        Male   69             0              1          Yes  Self-employed   
4      Female   37             0              0          Yes        Private   
5      Female   40             0              0          Yes        Private   
6        Male   35             0              0           No        Private   
7      Female   20             0              0           No        Private   

index Residence_type  avg_glucose_level   bmi   smoking_status  stroke  \
0              Urban             102.87  27.2  formerly smoked       1   
1              Rural             104.12  23.5     never smoked       1   
2              Rural 

In [34]:
print("prediction usinsg Linear SVC:\n")

testingSVCdf = testData 
testingSVCdf['predicted_stroke']=outputPredictionSVC
print(testingSVCdf)
SVCAccuracy = format( linearSvc.score(inputTestAt, outputTestAt))
print("\n")
print("Accuracy: %s"% SVCAccuracy)

prediction usinsg Linear SVC:

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   71             0              0          Yes        Private   
1        Male   80             0              0          Yes  Self-employed   
2      Female   65             0              0          Yes        Private   
3        Male   69             0              1          Yes  Self-employed   
4      Female   37             0              0          Yes        Private   
5      Female   40             0              0          Yes        Private   
6        Male   35             0              0           No        Private   
7      Female   20             0              0           No        Private   

index Residence_type  avg_glucose_level   bmi   smoking_status  stroke  \
0              Urban             102.87  27.2  formerly smoked       1   
1              Rural             104.12  23.5     never smoked       1   
2              Rural             10

In [35]:
print("prediction usinsg BernoulliNB:\n")

testingNBdf = testData 
testingNBdf['predicted_stroke']=outputPredictionNB
print(testingNBdf)
nbAccuracy = format( bernoulliNB.score(inputTestAt, outputTestAt))
print("\n")
print("Accuracy: %s"% nbAccuracy)

prediction usinsg BernoulliNB:

index  gender  age  hypertension  heart_disease ever_married      work_type  \
0        Male   71             0              0          Yes        Private   
1        Male   80             0              0          Yes  Self-employed   
2      Female   65             0              0          Yes        Private   
3        Male   69             0              1          Yes  Self-employed   
4      Female   37             0              0          Yes        Private   
5      Female   40             0              0          Yes        Private   
6        Male   35             0              0           No        Private   
7      Female   20             0              0           No        Private   

index Residence_type  avg_glucose_level   bmi   smoking_status  stroke  \
0              Urban             102.87  27.2  formerly smoked       1   
1              Rural             104.12  23.5     never smoked       1   
2              Rural             1

# Selection of Best Model

# Combine Data (Train+Test)

In [36]:
print("Train features in the form of DataFrame\n")

print(fTrainDatadf)

Train features in the form of DataFrame

index  gender  age  hypertension  heart_disease  ever_married  work_type  \
0           1   67             0              1             1          1   
1           1   80             0              1             1          1   
2           0   49             0              0             1          1   
3           0   79             1              0             1          2   
4           1   81             0              0             1          1   
5           1   74             1              1             1          1   
6           0   69             0              0             0          1   
7           0   78             0              0             1          1   
8           0   81             1              0             1          1   
9           0   61             0              1             1          0   
10          0   54             0              0             1          1   
11          0   79             0              1

In [37]:
print("Test features in the form of DataFrame\n")

print(fTestDatadf)

Test features in the form of DataFrame

index  gender  age  hypertension  heart_disease  ever_married  work_type  \
0           1   71             0              0             1          0   
1           1   80             0              0             1          1   
2           0   65             0              0             1          0   
3           1   69             0              1             1          1   
4           0   37             0              0             1          0   
5           0   40             0              0             1          0   
6           1   35             0              0             0          0   
7           0   20             0              0             0          0   

index  Residence_type  avg_glucose_level   bmi  smoking_status  stroke  
0                   1             102.87  27.2               0       1  
1                   0             104.12  23.5               1       1  
2                   0             100.98  28.2          

In [38]:
print("All features in the form of DataFrame\n")

alldf=[fTrainDatadf,fTestDatadf]
finaldf=pd.concat(alldf)
print(finaldf)

All features in the form of DataFrame

index  gender  age  hypertension  heart_disease  ever_married  work_type  \
0           1   67             0              1             1          1   
1           1   80             0              1             1          1   
2           0   49             0              0             1          1   
3           0   79             1              0             1          2   
4           1   81             0              0             1          1   
5           1   74             1              1             1          1   
6           0   69             0              0             0          1   
7           0   78             0              0             1          1   
8           0   81             1              0             1          1   
9           0   61             0              1             1          0   
10          0   54             0              0             1          1   
11          0   79             0              1  

# Train Best Model on All Data

In [39]:
allInputAt=finaldf.iloc[:, :-1].values
allOutputAt=finaldf.iloc[:, 10].values

In [40]:
randomForest.fit(allInputAt,allOutputAt)

RandomForestClassifier()

# Save the Trained Model as Pickle File

In [41]:
filename='bestModel.sav'
pickle.dump(randomForest,open(filename,'wb'))

# Make Predictions on Unseen/New Data

# Load the Trained Model

In [42]:
loadBM=pickle.load(open(filename,'rb'))


# Take Input from User

In [57]:
Gender=input("Please enter your gender:")
age=input("Please enter your age here :")
hypertension=input("Pleas eenter your hypertension(0/1):")
heart_disease=input("Do you have heart disease (1/0):")
ever_married=input("Do you ever married (Yes/No):")
work_type=input("Please enter your work type here (Private/Self-employed/Govt_job):")
Residence_type=input("Please enter your Residence type (Urban/Rural):")
avg_glucose_level=input("Please enter your avg glucose level:")
bmi=input("Please enter your bmi(20-50):")
smoking_status=input("Do you smoke (formerly smoked/never smoked/smokes/Unknown):")

Please enter your gender:male
Please enter your age here :23
Pleas eenter your hypertension(0/1):0
Do you have heart disease (1/0):0
Do you ever married (Yes/No):no
Please enter your work type here (Private/Self-employed/Govt_job):private
Please enter your Residence type (Urban/Rural):rural
Please enter your avg glucose level:100
Please enter your bmi(20-50):30
Do you smoke (formerly smoked/never smoked/smokes/Unknown):smokes


# Convert User Input into Feature Vector

In [58]:
print("user input in actual DataFrame: \n")
newdf=pd.DataFrame({"Gender":Gender,"Age":age,"Hypertension":hypertension,"Heart_disease":heart_disease,"Ever_married":ever_married,"Work_type":work_type,"Residence_type":Residence_type,"Avg_glucose_level":avg_glucose_level,"Bmi":bmi,"Smoking_status":smoking_status},index=[0])
print(newdf)

user input in actual DataFrame: 

  Gender Age Hypertension Heart_disease Ever_married Work_type Residence_type  \
0   male  23            0             0           no   private          rural   

  Avg_glucose_level Bmi Smoking_status  
0               100  30         smokes  


In [59]:
print("User input in Encoded DataFrame: \n")

if  Gender == "male" or Gender == "Male":
    Gender=1
elif  Gender=="female" or Gender=="Female":
      Gender=0

if  ever_married == "no" or ever_married == "No":
    ever_married=0
elif  ever_married=="yes" or ever_married=="Yes":
      ever_married=1

    
if work_type=="private" or work_type=="Private":
    work_type=1
elif work_type=="self-employed" or work_type=="Self-employed":
    work_type=2
elif work_type=="govt_job" or work_type=="Govt_job":
    work_type=0
    
if Residence_type=="urban" or Residence_type=="Urban":
    Residence_type=1
elif Residence_type=="rural" or Residence_type=="Rural":
    Residence_type=0

if smoking_status=="formerly smoked" or smoking_status=="formerly smoked":
    smoking_status=1
elif smoking_status=="never smoked" or smoking_status=="never smoked":
    smoking_status=2
elif smoking_status=="smokes" or smoking_status=="smokes":
    smoking_status=3
elif smoking_status=="Unknown" or smoking_status=="Unknown":
    smoking_status=0
encodeNewdf=pd.DataFrame({"Gender":Gender,"Age":age,"Hypertension":hypertension,"Heart_disease":heart_disease,"Ever_married":ever_married,"Work_type":work_type,"Residence_type":Residence_type,"Avg_glucose_level":avg_glucose_level,"Bmi":bmi,"Smoking_status":smoking_status},index=[0])

print(encodeNewdf)

User input in Encoded DataFrame: 

   Gender Age Hypertension Heart_disease  Ever_married  Work_type  \
0       1  23            0             0             0          1   

   Residence_type Avg_glucose_level Bmi  Smoking_status  
0               0               100  30               3  


In [60]:
print("User input in actual DataFrame: \n")
print(newdf)
print("\n \n \nUser input in encoded DataFrame \n")
print(encodeNewdf)

User input in actual DataFrame: 

  Gender Age Hypertension Heart_disease Ever_married Work_type Residence_type  \
0   male  23            0             0           no   private          rural   

  Avg_glucose_level Bmi Smoking_status  
0               100  30         smokes  

 
 
User input in encoded DataFrame 

   Gender Age Hypertension Heart_disease  Ever_married  Work_type  \
0       1  23            0             0             0          1   

   Residence_type Avg_glucose_level Bmi  Smoking_status  
0               0               100  30               3  


# Apply Trained Model on Feature Vector of Unseen Data and Output Prediction to User

In [61]:
outputPredictionBM= loadBM.predict(encodeNewdf)

if outputPredictionBM==0:
    stroke= "Person has no stroke"
    
elif outputPredictionBM==1:
    stroke="Person has stroke"

print("\n Prediction: %s"%stroke)



 Prediction: Person has no stroke
